In [46]:
#Opening the matching catalogs from TopCat for later comparison
from astropy.io.votable import is_votable, parse

def is_VOtable(fullname):
    tag = is_votable(fullname)
    print("The file", fullname, "is a VOtable, right ?", tag)
    return tag

pathdata = "outputs/"
data     = ["matching_morpho_acc_0.001_deg.vot", "matching_morpho_acc_0.01_deg.vot", 
            "matching_morpho_acc_1_arcsec.vot"]

#Checking that file format is correct
for name in data:
    voTag = is_VOtable(pathdata+name)
    if voTag:        
        fullFileName = pathdata + name
        #Retrieving the data
        table = parse(fullFileName)
        full  = table.get_first_table()
        
        print("Size of", name, "is", full.array.shape[0], "\n")
    else:
        exit("Exiting")

The file outputs/matching_morpho_acc_0.001_deg.vot is a VOtable, right ? True
Size of matching_morpho_acc_0.001_deg.vot is 372 

The file outputs/matching_morpho_acc_0.01_deg.vot is a VOtable, right ? True
Size of matching_morpho_acc_0.01_deg.vot is 481 

The file outputs/matching_morpho_acc_1_arcsec.vot is a VOtable, right ? True
Size of matching_morpho_acc_1_arcsec.vot is 305 



In [26]:
#Matching with astropy
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

In [21]:
field_gals  = pathdata+"master_field_galaxiesV2.vot"
morpho_gals = ["morphology/cosmos_morph_cassata_1.1.vot", "morphology/cosmos_morph_tasca_1.1.vot"]

#Checking file format
is_VOtable(field_gals)
for name in morpho_gals:
    is_VOtable(name)

The file outputs/master_field_galaxiesV2.vot is a VOtable, right ? True
The file morphology/cosmos_morph_cassata_1.1.vot is a VOtable, right ? True
The file morphology/cosmos_morph_tasca_1.1.vot is a VOtable, right ? True


In [24]:
#Retrieving data
field   = parse(field_gals).get_first_table()
morph_c = parse(morpho_gals[0]).get_first_table()
morph_t = parse(morpho_gals[1]).get_first_table()

field_RA, field_Dec     = np.array(field.array[['RA', 'DEC']].tolist()).T
morph_c_RA, morph_c_Dec = np.array(morph_c.array[['RA', 'DEC']].tolist()).T
morph_t_RA, morph_t_Dec = np.array(morph_t.array[['RA', 'DEC']].tolist()).T

In [47]:
field_catalog   = SkyCoord(ra=field_RA*u.degree, dec=field_Dec*u.degree)
morph_c_catalog = SkyCoord(ra=morph_c_RA*u.degree, dec=morph_c_Dec*u.degree)
morph_t_tatalog = SkyCoord(ra=morph_t_RA*u.degree, dec=morph_t_Dec*u.degree)

indices_c, dist2D_c, dist3D_c = field_catalog.match_to_catalog_sky(morph_c_catalog)

In [68]:
#Checking that the catalog gives consistent results with the former array when applying the mask
print("The former and later arrays are identical, right ?", not np.any(np.logical_not(morph_c.array[indices_c]['RA'] == morph_c_catalog[indices_c].ra.degree)))
morph_c.array[indices_c]['RA'] == morph_c_catalog[indices_c].ra.degree

The former and later arrays are identical, right ? True


masked_array(data=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,

In [73]:
#Checking that the index array as the same dimension as the field galaxies array
indices_c.shape[0] == field.array.shape[0]

True